load libraries

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.callbacks import *
import itertools

Using TensorFlow backend.


In [2]:
dir = "/ext/Data/aichallenger/scene/"

model_image_size = (299, 299)
fine_tune_layer = 173
final_layer = 314
visual_layer = 311
batch_size = 128

classdf = pd.read_csv("scene_classes.csv")
for i in range(80):
    if i % 10 == 9:
        print(classdf.loc[i]["chinese"])

橄榄球场
拳击场
塔
会议室
商店
舞厅
沟渠
雪屋/冰雕(山)


# 加载数据集

load train data

In [3]:
train_gen = ImageDataGenerator(
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
#     rotation_range=10.,
#     width_shift_range=0.05,
#     height_shift_range=0.05,
#     shear_range=0.1,
#     zoom_range=0.1,
)
gen = ImageDataGenerator(
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
)

classes = []
for i in range(80):
    classes.append(str(i))
train_generator = train_gen.flow_from_directory(os.path.join(dir, 'train'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical", classes=classes)
print("subdior to train type {}".format(train_generator.class_indices))
valid_generator = gen.flow_from_directory(os.path.join(dir, 'valid'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical", classes=classes)
print("subdior to valid type {}".format(valid_generator.class_indices))

Found 53879 images belonging to 80 classes.
subdior to train type {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '21': 21, '22': 22, '23': 23, '24': 24, '25': 25, '26': 26, '27': 27, '28': 28, '29': 29, '30': 30, '31': 31, '32': 32, '33': 33, '34': 34, '35': 35, '36': 36, '37': 37, '38': 38, '39': 39, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '50': 50, '51': 51, '52': 52, '53': 53, '54': 54, '55': 55, '56': 56, '57': 57, '58': 58, '59': 59, '60': 60, '61': 61, '62': 62, '63': 63, '64': 64, '65': 65, '66': 66, '67': 67, '68': 68, '69': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79}
Found 7120 images belonging to 80 classes.
subdior to valid type {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 1

# 构建模型

https://github.com/fchollet/keras/blob/master/keras/applications/inceptionV3.py


In [4]:
def make_model(optimizer, dropout, lr, tune_layer):
    input_tensor = Input((*model_image_size, 3))
    x = input_tensor
    x = Lambda(inception_v3.preprocess_input)(x)

    base_model = InceptionV3(input_tensor=x, weights='imagenet', include_top=False)

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(dropout)(x)
    x = Dense(80, activation='softmax')(x)
    model = Model(base_model.input, x)

    print("total layer count {}".format(len(base_model.layers)))

    for i in range(tune_layer):
        model.layers[i].trainable = False
        
    if optimizer == "adam":
        optimizer_class = Adam(lr=lr)
    elif optimizer == "rmsprop":
        optimizer_class = RMSprop(lr=lr)
        
    model.compile(optimizer=optimizer_class, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 训练模型

In [5]:
print("train_generator.samples = {}".format(train_generator.samples))
print("valid_generator.samples = {}".format(valid_generator.samples))
steps_train_sample = train_generator.samples // 128 + 1
steps_valid_sample = valid_generator.samples // 128 + 1

train_generator.samples = 53879
valid_generator.samples = 7120


In [6]:
import itertools

optimizers = ['adam','rmsprop']
dropouts = [0,0.25,0.5]
lrs = [0.001, 0.0001, 0.00001]
tune_layers = [fine_tune_layer,  213, 253]

parameters = itertools.product(optimizers, dropouts, lrs, tune_layers)

filepath="models/{epoch:03d}- Loss:{val_loss:.3f}.h5"
callbacks = [EarlyStopping(monitor='val_loss',patience=6),ModelCheckpoint(filepath, monitor='val_loss',save_best_only=True)]     

for p in parameters:
    name = "optimizer,{}-dropout,{}-lr,{}-tune_layer,{}.h5".format(p[0],p[1],p[2],p[3])
    print(name)
    model = make_model(p[0],p[1],p[2],p[3])
    model.fit_generator(train_generator, 
                                                  steps_per_epoch=steps_train_sample, 
                                                  validation_data=valid_generator, 
                                                  validation_steps=steps_valid_sample, 
                                                  callbacks=callbacks)
    model.save("models/" + name)
print("model saved!")

optimizer,adam-dropout,0-lr,0.001-tune_layer,173.h5
total layer count 312
Epoch 1/1


/home/jidou/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:514: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


421/421 [==============================] - 1118s - loss: 1.4252 - acc: 0.6084 - val_loss: 1.1821 - val_acc: 0.6610

ResourceExhaustedError: OOM when allocating tensor with shape[128,17,17,192]
	 [[Node: batch_normalization_2597/moments/sufficient_statistics/SquaredDifference = SquaredDifference[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](conv2d_2597/convolution, batch_normalization_2597/moments/StopGradient)]]
	 [[Node: batch_normalization_2621/moments/sufficient_statistics/Gather/_108985 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_6928_batch_normalization_2621/moments/sufficient_statistics/Gather", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'batch_normalization_2597/moments/sufficient_statistics/SquaredDifference', defined at:
  File "/home/jidou/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jidou/anaconda3/lib/python3.6/runpy.py", line 8

# 可视化模型

https://keras.io/visualization/

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import *

model = load_model("models/inceptionV3-imagenet-finetune{}.h5".format(fine_tune_layer))
print("load successed")

#SVG(model_to_dot(model).create(prog='dot', format='svg'))

# CAM 可视化

http://cnnlocalization.csail.mit.edu/

![](http://cnnlocalization.csail.mit.edu/framework.jpg)

$cam = (P-0.5)*w*output$

* cam: 类激活图 X\*X
* P: 概率
* output: 卷积层的输出 2048\*1
* w: 卷积核的权重 X\*X\*2048

In [ ]:
z = zip([x.name for x in model.layers], range(len(model.layers)))
for k, v in z:
    print("{} - {}".format(k,v))

http://docs.opencv.org/trunk/d3/d50/group__imgproc__colormap.html

![](http://docs.opencv.org/trunk/colorscale_jet.jpg)

In [ ]:
import matplotlib.pyplot as plt
import random
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

def show_heatmap_image(model_show, weights_show):
    test_dir = os.path.join(dir,  "test", "test" )
    image_files = glob.glob(os.path.join(test_dir,"*"))
    print(len(image_files))
    
    plt.figure(figsize=(12, 21))
    for i in range(15):
        plt.subplot(5, 3, i+1)
        img = cv2.imread(image_files[200*i+13])
        img = cv2.resize(img,  (model_image_size[1],model_image_size[0]))
        x = img.copy()
        x.astype(np.float32)
        out, predictions = model_show.predict(np.expand_dims(x, axis=0))
        predictions = predictions[0]
        out = out[0]
        
        max_idx = np.argmax(predictions)
        prediction = predictions[max_idx]

        plt.title('c%d |%s| %.2f%%' % (max_idx , classdf.loc[max_idx]["english"], prediction*100))
    
        cam = (prediction - 0.5) * np.matmul(out, weights_show)
        cam = cam[:,:,max_idx]
        cam -= cam.min()
        cam /= cam.max()
        cam -= 0.2
        cam /= 0.8

        cam = cv2.resize(cam, (model_image_size[1],model_image_size[0]))
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam <= 0.2)] = 0

        out = cv2.addWeighted(img, 0.8, heatmap, 0.4, 0)

        plt.axis('off')
        plt.imshow(out[:,:,::-1])
print("done")

In [ ]:
weights = model.layers[final_layer].get_weights()[0]
layer_output = model.layers[visual_layer].output
model2 = Model(model.input, [layer_output, model.output])
print("layer_output {0}".format(layer_output))
print("weights shape {0}".format(weights.shape))
show_heatmap_image(model2, weights)

In [ ]:
import json

def gen_test_result(model,  model_image_size, json_name):
    gen = ImageDataGenerator()
    test_generator = gen.flow_from_directory(dir + "test/",  model_image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)

    y_pred = model.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)
    print("y_pred shape {}".format(y_pred.shape))
    print(y_pred[0])

    l = list()
    for i, fname in enumerate(test_generator.filenames):
        name = fname[fname.rfind('/')+1:]
        d = dict()
        d["image_id"] = name
        d["label_id"] = y_pred[i].argsort()[-3:][::-1].tolist()
        l.append(d)
   
    json.dump(l, open(json_name, "w"))
    print("json saved")

print("done")

In [ ]:
gen_test_result(model,  model_image_size, 'json/InceptionV3-imagenet-finetune{}-pred.json'.format(fine_tune_layer))